## Model Training Challenge! 

In this notebook, there are a whole host of weird errors. 
Some of them will be obvious and throw an error, but some are less clear and will break things without telling you they're breaking. 
Do your best to sniff out the mistakes in both code and training procedure! 

Your mission, should you choose to accept it, is: 
* Work together in groups of 4 to complete this notebook
* Figure out all the coding mistakes and make a notebook that runs
* Find the mistakes in training and take appropriate corrective measures so the model trains well!
* Make a ~15 minute long presentation showing your results and the steps you took to find and solve them.

### Rules 
* If you cannot correct the problem, identify it and write a quick paragraph about what you would do
* You are allowed to use any resource you can find; other groups, TA's, a random scientist walking by, and most importantly, the internet. The use of generative AI is highly discouraged - not because it's cheating, but because it can send you down rabbit holes that are hard to find your way out of. 

In [ ]:
# Installing the data generation package 
! pip install deepbench

In [ ]:
! curl -o challenge_utilities.py https://raw.githubusercontent.com/BNL-Fermilab-RENEW/tutorials_2024/tree/main/07_Challenge/challenge_utilities.py

In [ ]:
# Now we'll import the different classes from this new file 
from challenge_utilities import SkyGenerator04 as SkyGenerator # `as` renames the imported package name
from challenge_utilities import Eval

In [ ]:
# Standard packages 

import numpy as np
import matplotlib.pyplot as plt

from tensorflow.keras.layers import Input, Dropout, Conv1D, Dense, AvgPool1D, Flatten
from tensorflow.keras.models import Model

import tensorflow as tf 
from sklearn.preprocessing import MinMaxScaler

#### Package documentation 

If you get stuck for syntax or anything - these are the packages used. 
Look up a function you're trying to use in their package search pages, and see if you maybe have types wrong, or wrong variable names. 

[Numpy]()

[MatPlotLib]() 

[Tensorflow/tf/Keras]()

[Sci-kit Learn]()

In [ ]:
label_map = {
    0: "Star", 
    1: "Galaxy"
}


def plot_samples(generator, n_columns=3, n_rows=3, label_map=None): 
    _, subplots = plt.subplots(n_columns, 1) # Make 9 plots in a 3 x 3 grid 
    plt.tight_layout()
    plt.setp(subplots, xticks=[], yticks=[])

    for sample_index, subplot in zip(range(n_columns*n_rows), subplots.ravel()): 
        image, label = generator[sample_index]
        subplot.imshow(image.squeeze()) 
        # 'imshow' displays an image in 2d (black and white if it only has 1 color channel, or in color if it has 3 (r,g,b) color channels). 
        # Here it's green and blue because the default colorway for matplotlib is "viridis", with is all cool colors
        
        string_label = "??" 
        subplot.set_xlabel(string_label) # This gives you a label underneath the image (on the x axis)

samples = SkyGenerator(n_samples=9, batch_size=1) # Can just get a few samples
plot_samples(samples)

# Look at the input data 

Understanding the data is a critical part of the training process, let's take a look at the distributions we're working with. 

In [ ]:
n_train_samples = 1280
train_generator = SkyGenerator(n_samples=n_train_samples, shuffle=True)

n_val_samples = 1280
val_generator = SkyGenerator(n_samples=n_val_samples, train=False, shuffle=True)

In [ ]:
# We can look at the distribution of labels by grabbing them from the generator 
    # TF.Sequence generators supply data as a tuple 
    # of (x,y) (or (features, labels)) 
    # - so using index 1 we can get the labels

all_train_labels = np.array([
    train_generator[i][1] for i in range(len(train_generator))
]).ravel()

all_val_labels = np.array([
    val_generator[i][0] for i in range(len(val_generator))
]).ravel()

figure, subplots = plt.subplots(1, 2)
subplots.hist(all_train_labels)
subplots[0].set_xlabel("Train")

subplots[1].hist(all_val_labels)
subplots[1].set_xlabel("Validation")

figure.supxlabel("Label Distributions")
figure.supylabel("Label Frequency")

plt.show()

In [ ]:
# We can also look at different attributes of the images 
# Here, let's look at max values and mean values

# These numbers will be slightly different every time, there's randomness controlling how the images are generated 
# In scientific settings, it's good to remove the randomness if possible, but here it's fine

all_train_labels = np.array([
    train_generator[i][0].ravel().mean() for i in range(len(train_generator))
]).ravel()

all_val_labels = np.array([
    val_generator[i][0].mean() for i in range(len(val_generator))
]).ravel()

figure, subplots = plt.subplots(1, 2)
subplots[0].hist(all_train_labels)
subplots[0].set_xlabel("Train")

subplots[1].hist(all_val_labels)
subplots[1].set_xlabel("Validation")

figure.suptitle("Mean value of pixels in a single image")
plt.show()


all_train_labels = np.array([
    train_generator[i][0].ravel().max() for i in range(len(train_generator))
]).ravel()

all_val_labels = np.array([
    val_generator[i][0].ravel().max() for i in range(len(val_generator))
]).ravel()

figure, subplots = plt.subplots(1, 2)
subplots[0].hist(all_train_labels)
subplots[0].set_xlabel("Train")

subplots[1].hist(all_val_labels)
subplots[1].set_xlabel("Validation")

figure.suptitle("Max value of pixels in a single image")
plt.show()

## Data Processing

There are many ways to help a model along when it comes to training - one of those is data pre-processing.
There are near infinite ways to pre-process, especially in the computer vision space. 
Think about it like applying different instagram filters, they have different impacts that emphasis the image in different ways. 

For this, let's try bringing all the pixels in the image between 0 and 1. 

In [ ]:
scaler = MinMaxScaler((0, 1))

fit_data = np.concatenate([train_generator[i][0] for i in range(5)], axis=0) 
# This is 5 batches of data, being used to find the approximate min and max of the data
# Because our data is synthetic, we don't need to worry about big outliers

# Unfortunately, minmaxscaler only handles 1d of data, so we need to do a little pre-and-post processing on the input 
# combining the last two dimensions together, making the 2d of the image 1d instead. 
fit_data = fit_data.reshape((fit_data.shape[0], fit_data.shape[1]*fit_data.shape[2]) ) 
scaler_fit = scaler.fit(fit_data)

def processor(image): 
    # This function will take a single image and return the scaled version 
    image_flat = image.ravel() 
    image_scaled = scaler_fit.transform(image_flat.reshape(1, -1))
    # Magically reshape it back into 2d 
    image_scaled_reshaped = image_scaled.reshape(image.shape)
    return image_scaled_reshaped

In [ ]:
# Look at the samples again! 

n_samples = 9
samples = SkyGenerator(n_samples=n_samples, batch_size=1, pre_processing=processor)
plot_samples(samples)

## Make the binary classification model 

This function, when called, produces a keras Model instance that you can train to predict a class of an input. 
Because this is a binary predictor, it can be used to pick if an image is closer to being class 0 or class 1. 
It takes an input of a certain shape, (defined by the `in_layer`), fits it to a convolution operation, and gives you a number (or array!) back out. 
The way this becomes a predictive engine is through the loss, of the output of the model will minimize a loss function, and give us a prediction that matches the data we fed it. 

In this case, what we want: 
* Take the input images from the data generator 
* Apply two convolutional blocks to the input image 
* Decode the second convolution block's output to a probability of the image being a given class. 

In [ ]:
def make_model(): 
    """
    Make a network that can perform binary classification

    Returns:
        model (keras.Model): classifier model that will 
    """
    in_layer = Input((28, 28))
    x = Conv1D(filters=4, kernel_size=2)(input_layer)
    x = Conv1D(filters=8, kernel_size=4)(x)
    x = Conv1D(filters=12, kernel_size=6)(x)
    
    x = AvgPool1D(6)(x)

    x = Conv1D(filters=4, kernel_size=2)(x)
    x = Conv1D(filters=8, kernel_size=4)(x)
    x = Conv1D(filters=12, kernel_size=6)(x)

    x = Dense(20, activation='relu')
    x = AvgPool1D(6)(x)

    x = Flatten()(x)

    x = Dropout(0.3)(x)
    output = Dense(10, activation='sigmoid')(x)
    model = Model(in_layer, x)
    
    return model

model = make_model()
loss = tf.losses.BinaryCrossropy()
optimizer = tf.keras.optimizers.aDAM(0.01)

# Compile tells the keras backend what loss and optimizer to use to perform gradients on the model
# You cannot train a keras model without compiling it first
model.compile(loss=loss, optimizer=optimizer)

# Show what layers are in the model, and their input and output shapes 
# This can help make sure all your stuff is a correct size
model.summary()

# Train the model 

We have all the pieces in place:
- [x] Model 
- [x] Train Data 
- [x] Validation Data 
- [x] Loss Function 
- [x] Optimizer 

Now lets put this together into a fit model. 
Keras trains in place, so you don't need a new variable to hold the `fit_model` vs `model`. 
Once you call `fit`, the model is fit, and it re-train, you need to make a new model with the `make_model()` function. 

In [ ]:
train_generator_scaled = SkyGenerator(n_samples=1280, pre_processing=processor, batch_size=64)
val_generator_scaled = SkyGenerator(n_samples=12, pre_processing=None, batch_size=64)

history = model.fit(
    train_generator_scaled, 
    validation_data=train_generator_scaled, 
    epchs=1, 
    verbose=1
    ).history

## Model Evaluation 

There are some steps we can take to see how well a model trained. 

### Loss Plots 
Obvious one is to see how the loss progressed - if the loss was still trending down when the training stopped, it would make sense that the model would benefit from longer training. 
Or, if the loss never moves or blows up entirely, that's a sign there's a problem. 
Looking at the [common pitfalls notebook](https://github.com/BNL-Fermilab-RENEW/tutorials_2024/blob/main/07_Challenge/common_pitfalls.ipynb) may help diagnose your problems! 

In [ ]:
Eval.plot_loss_history(history) 
# Eval.plot_history is a simple function 
# plots the loss as a function of epoch 

## Classification Accuracy Plots 

After we did all this work to train a model, we need to be able to report how good it is on data we didn't use in training. 
For this, we'll make a new set of data (or use some data we held out from training), and run a few evaluation metrics on it. 

### ROC
The `receiver operating characteristic curve` (or just "ROC" (pronounced "Rock") Curve) is a metric that plots the true positive rate against the false positive rate. 
It shows how likely a model is to correctly predict something. 
The idea is that a classifier a better classifier will have lower false positive rate, and a higher true positive rate, so the curve will get closer and closer to the upper left corner as the prediction improves. 

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/1/13/Roc_curve.svg/800px-Roc_curve.svg.png" width="300">

<a href="https://upload.wikimedia.org/wikipedia/commons/3/36/Roc-draft-xkcd-style.svg"></a><a href="//commons.wikimedia.org/wiki/File:Roc-draft-xkcd-style.svg#filelinks" title="File:Roc-draft-xkcd-style.svg">Roc-draft-xkcd-style.svg</a>, <a href="https://creativecommons.org/licenses/by-sa/4.0" title="Creative Commons Attribution-Share Alike 4.0">CC BY-SA 4.0</a>, <a href="https://commons.wikimedia.org/w/index.php?curid=109730045">Link</a>


### Confusion Matrix

Confusion matrices are a great tool for seeing how well each class does against each other. 
It gets it name from its ability to tell if a model is "confusing" two different classes. 
It plots the rate of predicted values for a given class versus the true values. 

<img src="https://www.ibm.com/content/dam/connectedassets-adobe-cms/worldwide-content/creative-assets/s-migr/ul/g/c8/a7/binary-matrix.component.complex-narrative-xl.ts=1712087356966.png/content/adobe-cms/us/en/topics/confusion-matrix/jcr:content/root/table_of_contents/body/content_section_styled/content-section-body/complex_narrative_390941229/items/content_group/image" width='400'>

 <a href="https://www.ibm.com/topics/confusion-matrix">Link</a>

 A good confusion matrix will have very high values in the green boxes, and lower values in the red boxes. 

In [ ]:
test_generator = SkyGenerator(n_samples=1280, train=False, shuffle=True)

def make_prediction(model, test_generator): 
    predictions = trained_model.predict(test_generator)
    prediction_classes = np.where(predictions<0.5, 0, 1)
    labels = test_generator.labels
    return prediction_classes, labels

def test_quality(prediction, labels): 
    accuracy = tf.keras.metrics.BinaryAccuracy()(labels, labels)
    return accuracy.numpy()

prediction, labels = make_prediction(model, test_generator)
print(f"The binary classification accuracy on the test set is: {test_quality(prediction, labels)}")

In [ ]:
# And now run them on your own data! 
Eval.ROC_curve(prediction, labels)
Eval.confusion_matrix(labels, labels)